<h1>Dask dataframe</h1>
<h3>Data Prep</h3>

In [2]:
%run prep_data.py -d flights

data_dir='/workspaces/Dask_library_tutorial/data'
- Downloading NYC Flights dataset... done
- Extracting flight data... done
** Created flights dataset! in 7.48s**


In [3]:
import os
files = os.path.join('data', 'flights', '*.csv')
files

'data/flights/*.csv'